In [10]:
import requests
import json
from time import sleep
from multiprocessing import Pool
import numpy as np
import pickle as pkl

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [5]:
cd drive/MyDrive/MP2-bigdata/data

/content/drive/MyDrive/MP2-bigdata/data


In [12]:
#Information for Riot API
region =  ['americas', 'NA1']

apis_list= [
'RGAPI-16b91d0d-4426-4781-9613-75f93864016b',

'RGAPI-61cae079-3647-43e4-afdf-8d1607699414',

'RGAPI-210b57b7-f1b2-4416-ae34-e810867d3893',

'RGAPI-83abfef8-c6d4-4471-8ba8-cc7b4860a9cf',

'RGAPI-1d7bc8cb-86fd-4674-ad03-3fde05f595fa',

'RGAPI-e28e9659-a084-4b7f-8f6a-7d28e49f0cda',

'RGAPI-fb39860b-2152-495c-a620-a591da78d8f4',

'RGAPI-526a243b-4e49-4c95-bc24-9eb826e7265c',

'RGAPI-e924d415-1b00-468a-a1b7-d22c4dbb8215',

'RGAPI-c9b3edf9-4797-4c2b-ab7a-c02d45131cf9 '
]

len(apis_list)

10

# Match id collection

1- get a bunch of  summoner  from the league api.To have a realistic distribution of summoner ranks, we omit the challenger and grandmaster pool completely and add a single master summoner by hand to the dataset; since these higher ranks represent an extremely small percentage of players.

## summoners collection

In [ ]:

#Step 1: collect some ranked summoner IDs
#get a bunch of summoner IDs from the league-v4 api
def get_summoners(region,api_key,teir,dev, page):
    """ function to get ranked summoner ids from leagues"""    

    URL = f'https://{region}.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{teir}/{dev}?page={page}&api_key={api_key}'
    LeagueList = requests.get(URL).json()
    sleep(0.9)
    summonerNames = []
    for i in LeagueList:
        summonerNames.append(i['summonerName'])
    return set(summonerNames)




In [ ]:
from collections import defaultdict 
summonerNames = defaultdict(set)
teirs = [('BRONZE',11)  ,  ( 'SILVER',22) ,( 'GOLD',17) , ( 'DIAMOND',2)  ,( 'IRON',2) , ('PLATINUM',7)]

for i in teirs:
    rank = i[0]
    pages = i[1]
    if i == 'BRONZE' or i == 'IRON': d='I'
    else: d='IV'
    for page in range(1,pages):
        s = get_summoners(region[1],apis_list[0], rank,d, page)
        summonerNames[rank] = summonerNames[rank].union(s)


In [ ]:
total=0
for k, v in summonerNames.items():
    print(f'{k}:   {len(v)}    summoners')
    total+=len(v)

print(f"total: {total}")

BRONZE:   2005    summoners
SILVER:   4205    summoners
GOLD:   3205    summoners
DIAMOND:   205    summoners
IRON:   205    summoners
PLATINUM:   1205    summoners
total: 11030


In [ ]:
import pickle as pkl
with open('COMPLETE_DATA/summoners.pkl', 'wb') as f:
    pkl.dump(summonerNames, f)

In [ ]:
complete_summ_list = []
for k,v in summonerNames.items():
    complete_summ_list.extend(list(v))
len(complete_summ_list), len(set(complete_summ_list))

(11030, 11030)

In [ ]:
from random import shuffle
shuffle(complete_summ_list)
shuffle(complete_summ_list)
shuffle(complete_summ_list)

## get match ids from summoners

In [ ]:
match_ids = []

In [ ]:
#step2: convert summoner names to puuids using the v4-summoner api
def get_puuid(summ_name, api_key):
    """ function to get the puuid of a given summoner id"""
    URL = f'https://{region[1]}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summ_name}?api_key={api_key}'
    summ_info = requests.get(URL).json()
    # the delay prevents us from exceeding the maximum request rate per minute
    try: 
        summ_puuid= summ_info['puuid']
        return summ_puuid

    except: 
        print(summ_info, '\n', summ_name)
        return None
    
def match_by_puuid(puuid, api_key): 
    """ function to get match_id list for a given summoner puuid"""   
    URL = f'https://{region[0]}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?queue=420&type=ranked&start=0&api_key={api_key}'
    match_ids = requests.get(URL).json()
    #if type(match_ids) != list: print(f'error:   {match_id}\n' )
    return match_ids

In [ ]:
#Step 3: get the 20 most recent match_ids from all summoner puuids  using the v4-matches api
import numpy as np
def get_matches_by_summoner_names(args):
    summ_names, api_key = args[0], args[1]
    match_ids =[]
    for summ_name in summ_names:
        puuid = get_puuid(summ_name, api_key)
        if puuid:
            match_ids.extend(match_by_puuid(puuid, api_key))
    return match_ids
##########################
# loop over summoners to get match ids 
#match_ids = []

chunks = np.array_split(complete_summ_list[3500:], 10)
args = ([x, apis_list[i]] for i, x in enumerate(chunks))
p = Pool(10)
for r in p.imap_unordered(get_matches_by_summoner_names, args):
    match_ids.extend(r)
    #sleep(0.1)



In [ ]:
match_ids = list(set(match_ids))
shuffle(match_ids)
len(match_ids)

147396

In [ ]:
# now store the matchids in the dataset folder just in case
import pickle as pkl
with open('COMPLETE_DATA/140_thousand_match_ids.pkl', 'wb') as f:
    pkl.dump(set(match_ids), f)

## match object collection

In [6]:
import pickle as pkl
match_ids = pkl.load(open('COMPLETE_DATA/140_thousand_match_ids.pkl', 'rb'))
len(match_ids)

147396

In [7]:
match_objects = []
match_ids= list(match_ids)
match_ids[0]

'NA1_4099908864'

In [8]:
# step 3:  get match objects from match ids using the v4-matches api
def get_match_by_match_id(args):
    """ function to get the match object for a given match id"""
    match_ids, api_key = args[0], args[1]
    matches = []
    for match_id in match_ids:
        URL = f'https://{region[0]}.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'
        m = requests.get(URL).json()

        #print(m)

        sleep(1)
        try: 
            status = m['status']
            print(status)
            m=[]
        except KeyError:
            matches.append(m)
    
    return matches




#match_objects = [get_match_by_match_id(i, region[0], api_key) for i in my_segment]

In [ ]:
import numpy as np
chunks = np.array_split(list(match_ids)[:100500], 2)
args = ([x, apis_list[i]] for i, x in enumerate(chunks))
p = Pool(10)
for r in p.imap_unordered(get_match_by_match_id, args):

    match_objects.extend(r)


In [ ]:
cd COMPLETE_DATA

/content/drive/My Drive/MP2-bigdata/data/COMPLETE_DATA


In [ ]:
# store the data into a json file
with open('200_thousand_match_objects.pkl', 'wb') as f:
    pkl.dump(match_objects, f)


## format a json file and save

In [ ]:
pwd

'/content/drive/My Drive/MP2-bigdata/data/COMPLETE_DATA'

In [ ]:
with open('200_thousand_match_objects', 'a') as f:
    for i in match_objects:
        json.dump(i,f)
        f.write('\n')

In [ ]:
f = open('200_thousand_match_objects', 'r')
len(f.readlines())

201736